<a href="https://colab.research.google.com/github/nautim/digitalspacesciencespo/blob/main/project_predicting_ideology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = '/content/drive/MyDrive/digitalspace data/'

In [4]:
terror_nodups = pd.read_csv(path+'terror_nodups_new.csv.gz', compression='gzip', dtype='str', index_col=0)
terror_nodups = terror_nodups.drop(columns=['Unnamed: 0'])

In [ ]:
terror_nodups.head()

,id,author_id,created_at,edit_history_tweet_ids,lang,in_reply_to_user_id,conversation_id,text,reply_settings,possibly_sensitive,...,text_tokenized,text_lemmitized,tokens_merged,text_stemm,sentiment_vader,sentiment_textblob,sentiment_vader_cat,sentiment_textblob_cat,date,sentiment_tf
0,1604949964429426688,1257932018,2022-12-19 21:21:04+00:00,1604949964429426688,fr,NaN,1604949964429426688,RT @_BeFootball: Quel accueil pour Léo Messi à...,everyone,False,...,"['quel', 'accueil', 'léo', 'messi', 'retour', ...","['quel', 'accueil', 'léo', 'messi', 'retour', ...",quel accueil léo messi retour france,"['quel', 'accueil', 'léo', 'mess', 'retour', '...",0.0,0.0,neutral,neutral,2022-12-19,positive
1,1604949944984653849,1417719872331595780,2022-12-19 21:21:00+00:00,1604949944984653849,fr,361857853,1604060382779396099,@Autochtone_fr @Jager80568144 @DubhStephane @A...,everyone,False,...,"['puis', 'ne', 'faut', 'pas', 'oublier', 'fran...","['puis', 'ne', 'faut', 'pas', 'oublier', 'fran...",puis ne faut pas oublier france née migration,"['puis', 'ne', 'faut', 'pas', 'oubli', 'franc'...",0.1695,0.0,positive,neutral,2022-12-19,positive
2,1604949933508816896,1470318045314887683,2022-12-19 21:20:57+00:00,1604949933508816896,fr,NaN,1604949933508816896,RT @Marc_Claret75: 🔴Acceuil des migrants :nous...,everyone,False,...,"['acceuil', 'migrants', 'passés', 'foyer', 'ac...","['acceuil', 'migrant', 'passé', 'foyer', 'accu...",acceuil migrant passé foyer accueil centre rét...,"['acceuil', 'migr', 'pass', 'foi', 'accueil', ...",0.4019,-0.2,positive,negative,2022-12-19,negative
3,1604949930220847104,928722823,2022-12-19 21:20:56+00:00,1604949930220847104,fr,NaN,1604949930220847104,RT @Tajmaat_Service: 🇩🇿🇨🇵 FLASH - Un #Algérien...,everyone,False,...,"['flash', 'algérien', 'arrivé', 'france', 'âge...","['flash', 'algérien', 'arrivé', 'france', 'âge...",flash algérien arrivé france âge six an adam s...,"['flash', 'algérien', 'arriv', 'franc', 'âge',...",0.6369,0.21,positive,positive,2022-12-19,positive
4,1604949860993695746,2199917760,2022-12-19 21:20:40+00:00,1604949860993695746,fr,NaN,1604949860993695746,RT @chaine_verte: La Chaîne Verte est un group...,everyone,False,...,"['chaîne', 'verte', 'groupe', 'citoyens', 'fra...","['chaîne', 'verte', 'groupe', 'citoyen', 'fran...",chaîne verte groupe citoyen français européen ...,"['chaîn', 'vert', 'group', 'citoyen', 'franc',...",0.4391,0.152,positive,positive,2022-12-19,positive


In [5]:
terror_grading = pd.read_csv(path+'terror_grading.csv', dtype='str', index_col=0)

In [ ]:
terror_grading.head()

,author_id,created_at,text,sentiment,ideology
id,,,,,
1.383881e+18,2677366932,2021-04-18 20:30:06+00:00,@AlexVilp @dupontaignan Et l'impunité de la fl...,negative,left
1.607854e+18,1537825543885795330,2022-12-27 21:42:00+00:00,"RT @Tii_ech: @taetaeluvm #MohammadMoradi, un é...",negative,left
1.322590e+18,416409165,2020-10-31 17:24:12+00:00,@Fraoui1983 @JudithBernarADA @briceculturier @...,negative,left
1.320738e+18,766411072249204736,2020-10-26 14:43:33+00:00,Le voile a jamais tué personne en France c’est...,negative,left
1.319349e+18,2693196473,2020-10-22 18:46:36+00:00,@OlivierBenis @Alkanz Il faut un brassard à la...,negative,left


In [6]:
# ideological training
terror_grading['is_ideological'] = terror_grading['ideology'].map({'right': 1, 'left': 1, 'neutral': 0})
terror_grading['is_right'] = terror_grading['ideology'].map({'right': 1, 'left': 0, 'neutral': np.nan})


In [7]:
!git clone https://github.com/rubingshen/AugmentedSocialScientist.git
!pip install ./AugmentedSocialScientist/package/

from AugmentedSocialScientist import xlmroberta

Cloning into 'AugmentedSocialScientist'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 134 (delta 36), reused 50 (delta 13), pack-reused 35
Receiving objects: 100% (134/134), 1.68 MiB | 11.16 MiB/s, done.
Resolving deltas: 100% (46/46), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./AugmentedSocialScientist/package
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 44.6 MB/

In [8]:
data_train, data_test = xlmroberta.train_test_split(terror_grading, test_size=0.2, random_state=42, stratify=terror_grading['is_ideological'])


In [9]:
train_loader = xlmroberta.encode(data_train.text.values, data_train.is_ideological.values)
test_loader = xlmroberta.encode(data_test.text, data_test.is_ideological.values)

  0%|          | 0/1120 [00:00<?, ?it/s]

  0%|          | 0/1120 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

In [ ]:
# import torch
# torch.cuda.empty_cache()


In [10]:
score = xlmroberta.run_training(train_loader, 
                         test_loader, 
                         n_epochs=5, 
                         lr=5e-5, 
                         seed_val=42,
                         save_model_as='is_ideological')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p


======== Epoch 1 / 5 ========
Training...

  Average training loss: 0.67
  Training took: 0:00:11

Running Validation...

  Average test loss: 0.65
  Validation took: 0:00:01
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       104
           1       0.63      0.99      0.77       176

    accuracy                           0.62       280
   macro avg       0.31      0.49      0.38       280
weighted avg       0.39      0.62      0.48       280


======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.63
  Training took: 0:00:08

Running Validation...

  Average test loss: 0.61
  Validation took: 0:00:01
              precision    recall  f1-score   support

           0       0.63      0.38      0.48       104
           1       0.71      0.87      0.78       176

    accuracy                           0.69       280
   macro avg       0.67      0.63      0.63       280
weighted avg       0.68      0.69      0.67      

In [11]:
pred_loader = xlmroberta.encode(terror_nodups.text.values)

  0%|          | 0/118576 [00:00<?, ?it/s]

  0%|          | 0/118576 [00:00<?, ?it/s]

In [12]:
pred_proba = xlmroberta.predict_with_model(pred_loader, model_path='./models/is_ideological')

  0%|          | 0/3706 [00:00<?, ?it/s]

In [13]:
pred_proba = pd.DataFrame(pred_proba, columns=['not_ideological', 'ideological'])

In [14]:
pred_proba

,not_ideological,ideological
0,0.640445,0.359556
1,0.312288,0.687712
2,0.699077,0.300923
3,0.698919,0.301081
4,0.751740,0.248260
...,...,...
118571,0.211759,0.788241
118572,0.239549,0.760451
118573,0.525722,0.474278
118574,0.300241,0.699759


In [19]:
terror_nodups = pd.concat([terror_nodups, pred_proba], axis = 1)

Second training - for ideological tweets, if they are right or left.

In [34]:
terror_grading_ideological = terror_grading.dropna(subset = ['is_right'])

In [36]:
terror_grading_ideological.reset_index(drop=True, inplace=True)

In [37]:
data_train, data_test = xlmroberta.train_test_split(terror_grading_ideological, test_size=0.2, random_state=42, stratify=terror_grading_ideological['is_right'])

In [60]:
data_train.is_right = data_train.is_right.astype(int)

In [61]:
data_test.is_right = data_test.is_right.astype(int)

In [62]:
train_loader = xlmroberta.encode(data_train.text, data_train.is_right.values)
test_loader = xlmroberta.encode(data_test.text, data_test.is_right.values)

  0%|          | 0/703 [00:00<?, ?it/s]

  0%|          | 0/703 [00:00<?, ?it/s]

  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/176 [00:00<?, ?it/s]

In [72]:
score = xlmroberta.run_training(train_loader, 
                         test_loader, 
                         n_epochs=4, 
                         lr=5e-5, 
                         seed_val=42,
                         save_model_as='is_right')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.70
  Training took: 0:00:05

Running Validation...

  Average test loss: 0.68
  Validation took: 0:00:00
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        75
           1       0.57      1.00      0.73       101

    accuracy                           0.57       176
   macro avg       0.29      0.50      0.36       176
weighted avg       0.33      0.57      0.42       176


======== Epoch 2 / 4 ========
Training...


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat


  Average training loss: 0.69
  Training took: 0:00:05

Running Validation...

  Average test loss: 0.68
  Validation took: 0:00:00
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        75
           1       0.57      1.00      0.73       101

    accuracy                           0.57       176
   macro avg       0.29      0.50      0.36       176
weighted avg       0.33      0.57      0.42       176


======== Epoch 3 / 4 ========
Training...


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat


  Average training loss: 0.68
  Training took: 0:00:05

Running Validation...

  Average test loss: 0.68
  Validation took: 0:00:00
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        75
           1       0.57      1.00      0.73       101

    accuracy                           0.57       176
   macro avg       0.29      0.50      0.36       176
weighted avg       0.33      0.57      0.42       176


======== Epoch 4 / 4 ========
Training...


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat


  Average training loss: 0.69
  Training took: 0:00:05

Running Validation...

  Average test loss: 0.68
  Validation took: 0:00:00
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        75
           1       0.57      1.00      0.73       101

    accuracy                           0.57       176
   macro avg       0.29      0.50      0.36       176
weighted avg       0.33      0.57      0.42       176


Training complete!


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

Predictions of left/right are failing, so no sense to go further.